In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

import json
import requests

In [2]:
data_path = 'creditcard.csv'
df = pd.read_csv(data_path)

In [3]:
# querying with scaling
scaler = StandardScaler()
scaler.fit(df.drop(columns="Class", axis=1))

StandardScaler()

In [4]:
# transform data
scaled_selection = scaler.transform(df.iloc[:80].drop(["Class"], axis=1))

In [5]:
# select 80 values from your data frame to query your model with
# Create dictionary with pandas DataFrame in the split orientation
json_data = {"dataframe_split": pd.DataFrame(scaled_selection).to_dict(orient="split")}

In [6]:
# run the following command in Google Cloud Shell:
# mlflow models serve -m gs://ccfd-mlops-storage/mlruns/0/30316a6d471d41259cbd2910917e1593/artifacts/log_reg_model -h 10.142.0.2
# query the model from Google Cloud Virtual Machine through external IP address.
response = requests.post("http://35.196.11.25:5000/invocations", json=json_data)

In [7]:
preds = pd.DataFrame(response.json()['predictions'])
preds

,0
0,0
1,0
2,0
3,0
4,0
...,...
75,0
76,0
77,0
78,0


In [8]:
y_true = df.iloc[:80].Class
balanced_acc = balanced_accuracy_score(y_true, preds)

y_true.iloc[-1] = 1
f1score = f1_score(y_true, preds)

print("Balanced_acc", balanced_acc)
print("f1 score", f1score)

Balanced_acc 0.9375
f1 score 0.0


/Users/yingfan/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/var/folders/k1/cgm0wsnd5173q6n2ps293r080000gp/T/ipykernel_15501/153937322.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_true.iloc[-1] = 1


In [9]:
test = df.iloc[:8000]
true = test.Class
test = scaler.transform(test.drop(["Class"], axis=1))
preds = []

batch_size = 80
for f in range(100):
    print(f"Batch {f}", end=" - ")
    
    sample = pd.DataFrame(test[f*batch_size: (f+1)*batch_size]).to_dict(orient="split")
    
    json_data = {"dataframe_split": sample}
    response = requests.post("http://35.196.11.25:5000/invocations", json=json_data)
    resp = pd.DataFrame(response.json()['predictions'])
    preds = np.concatenate((preds, resp[0]))
    
    print("Completed")
    
balanced_acc = balanced_accuracy_score(true, preds)
f1score = f1_score(true, preds)

print("Balanced_acc", balanced_acc)
print("f1 score", f1score)

Batch 0 - Completed
Batch 1 - Completed
Batch 2 - Completed
Batch 3 - Completed
Batch 4 - Completed
Batch 5 - Completed
Batch 6 - Completed
Batch 7 - Completed
Batch 8 - Completed
Batch 9 - Completed
Batch 10 - Completed
Batch 11 - Completed
Batch 12 - Completed
Batch 13 - Completed
Batch 14 - Completed
Batch 15 - Completed
Batch 16 - Completed
Batch 17 - Completed
Batch 18 - Completed
Batch 19 - Completed
Batch 20 - Completed
Batch 21 - Completed
Batch 22 - Completed
Batch 23 - Completed
Batch 24 - Completed
Batch 25 - Completed
Batch 26 - Completed
Batch 27 - Completed
Batch 28 - Completed
Batch 29 - Completed
Batch 30 - Completed
Batch 31 - Completed
Batch 32 - Completed
Batch 33 - Completed
Batch 34 - Completed
Batch 35 - Completed
Batch 36 - Completed
Batch 37 - Completed
Batch 38 - Completed
Batch 39 - Completed
Batch 40 - Completed
Batch 41 - Completed
Batch 42 - Completed
Batch 43 - Completed
Batch 44 - Completed
Batch 45 - Completed
Batch 46 - Completed
Batch 47 - Completed
Ba